# 处理文本数据

## Step1 导包并测试版本

In [71]:
from importlib.metadata import version
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
from modelscope.msdatasets import MsDataset
import jieba
from torch.utils.data import Dataset , DataLoader
import torch
from torch import nn

torch version: 2.9.1
tiktoken version: 0.12.0


## Step2 导入数据集

In [2]:
#数据集下载
dataset = MsDataset.load('swift/Chinese-Qwen3-235B-2507-Distill-data-110k-SFT',cache_dir='./data')

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
print(dataset['train'][0])

{'messages': [{'role': 'user', 'content': '能给我讲一个寓意深刻的故事吗？'}, {'role': 'assistant', 'content': '当然可以。下面是一个寓意深刻的小故事，名为《石匠与神像》：\n\n很久以前，在一座深山里，有一位技艺精湛的石匠，名叫阿岩。他一生都在雕刻石头，作品精美绝伦，远近闻名。但他从不雕刻神像，别人问他为什么，他总是沉默不语。\n\n一天，村里要修建一座庙宇，村长请他雕刻一尊守护神像。阿岩犹豫了很久，终于答应了。他选了一块巨大的青石，开始日复一日地凿刻。每一锤、每一凿，他都极为专注，仿佛在与石头对话。\n\n三年过去了，神像终于完成。它面容庄严，目光深邃，仿佛能看透人心。村民们都惊叹不已，纷纷跪拜，称其为“活的神”。\n\n落成典礼那天，村长对阿岩说：“你创造了神，你就是神的化身。”\n\n阿岩却摇了摇头，轻声说：“不，我不是创造者，我只是唤醒了它。”\n\n众人不解。阿岩走到神像前，指着一处细微的裂痕说：“你们看，这块石头原本有一道裂纹，藏在深处。我本想避开它，可无论怎么雕，那裂痕总会出现在神像的眉心。最后我明白——它不是瑕疵，而是它本来的样子。”\n\n他顿了顿，望向群山：“这石头在山中沉睡了千年，经历了风雨雷电，才有了这道伤痕。我所做的，不是塑造完美，而是让伤痕也成为神圣的一部分。”\n\n众人沉默。\n\n阿岩又说：“人总想雕刻出完美的神，可真正的神，不是无瑕的偶像，而是包容伤痕、接纳残缺的存在。我们每个人心中都有一块石头，有裂痕，有沉重，有沉默。但正是这些，让我们真实，让我们值得被唤醒。”\n\n说完，他放下刻刀，走进山林，再也没有回来。\n\n——\n\n**寓意**：  \n这个故事告诉我们，真正的美与力量，不在于完美无缺，而在于接纳自己的伤痕与不完美。每个人都有裂痕，但那不是缺陷，而是经历的印记，是灵魂的深度。我们不必成为别人眼中的“完美神像”，而是要勇敢地“唤醒”真实的自己——带着伤痕，依然庄严站立。'}]}


## Step3 数据处理

In [4]:
import pandas as pd
def split_user_assistant(ds):
    user_rows = []
    assistant_rows = []
    train = ds['train'] if isinstance(ds, dict) and 'train' in ds else ds
    for item in train:
        if isinstance(item, dict):
            convo_key = None
            for k in ['conversations','messages','dialog','chat','conversation']:
                if k in item and isinstance(item[k], (list,tuple)):
                    convo_key = k
                    break
            if convo_key:
                for m in item[convo_key]:
                    role = m.get('role') if isinstance(m, dict) else None
                    content = m.get('content') if isinstance(m, dict) else (m[1] if isinstance(m,(list,tuple)) and len(m)>1 else None)
                    if role == 'user' and content is not None:
                        user_rows.append({'content': content})
                    elif role == 'assistant' and content is not None:
                        assistant_rows.append({'content': content})
                continue
            found = False
            for ukey, akey in [('instruction','output'),('prompt','response'),('question','answer'),('input','output'),('source','target'),('query','response'),('user','assistant')]:
                if ukey in item and akey in item:
                    user_rows.append({'content': item[ukey]})
                    assistant_rows.append({'content': item[akey]})
                    found = True
                    break
            if not found:
                pass
    user_df = pd.DataFrame(user_rows)
    assistant_df = pd.DataFrame(assistant_rows)
    return user_df, assistant_df
user_df, assistant_df = split_user_assistant(dataset)
user_list = user_df['content'].tolist()
assistant_list = assistant_df['content'].tolist()
print(len(user_list), len(assistant_list))


110000 110000


In [5]:
tokenizer = jieba
user_res = list(tokenizer.cut(user_list[0]))
user_res

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\19657\AppData\Local\Temp\jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built successfully.


['能', '给', '我', '讲', '一个', '寓意', '深刻', '的', '故事', '吗', '？']

In [6]:
from tqdm.auto import tqdm
user_res, assistant_res = [], []
for user, assistant in tqdm(zip(user_list, assistant_list), total=len(user_list)):
    user_res.append(list(tokenizer.cut(user)))
    assistant_res.append(list(tokenizer.cut(assistant)))
user_res[0], assistant_res[0]


100%|██████████| 110000/110000 [06:38<00:00, 276.36it/s]


(['能', '给', '我', '讲', '一个', '寓意', '深刻', '的', '故事', '吗', '？'],
 ['当然',
  '可以',
  '。',
  '下面',
  '是',
  '一个',
  '寓意',
  '深刻',
  '的',
  '小',
  '故事',
  '，',
  '名为',
  '《',
  '石匠',
  '与',
  '神像',
  '》',
  '：',
  '\n',
  '\n',
  '很久以前',
  '，',
  '在',
  '一座',
  '深山',
  '里',
  '，',
  '有',
  '一位',
  '技艺',
  '精湛',
  '的',
  '石匠',
  '，',
  '名叫',
  '阿岩',
  '。',
  '他',
  '一生',
  '都',
  '在',
  '雕刻',
  '石头',
  '，',
  '作品',
  '精美绝伦',
  '，',
  '远近闻名',
  '。',
  '但',
  '他',
  '从不',
  '雕刻',
  '神像',
  '，',
  '别人',
  '问',
  '他',
  '为什么',
  '，',
  '他',
  '总是',
  '沉默不语',
  '。',
  '\n',
  '\n',
  '一天',
  '，',
  '村里',
  '要',
  '修建',
  '一座',
  '庙宇',
  '，',
  '村长',
  '请',
  '他',
  '雕刻',
  '一尊',
  '守护神',
  '像',
  '。',
  '阿岩',
  '犹豫',
  '了',
  '很',
  '久',
  '，',
  '终于',
  '答应',
  '了',
  '。',
  '他选',
  '了',
  '一块',
  '巨大',
  '的',
  '青石',
  '，',
  '开始',
  '日复一日',
  '地凿刻',
  '。',
  '每',
  '一锤',
  '、',
  '每一凿',
  '，',
  '他',
  '都',
  '极为',
  '专注',
  '，',
  '仿佛',
  '在',
  '与',
  '石头',
  '对话',
  '。',
  '\n',
  '\n',
  '三年

In [10]:
import json
with open(r'G:\Anaconda\Kaggle\LLMs-from-scratch-main\my_test_ch01\data\tokenized_results.json', 'w', encoding='utf-8') as f:
    json.dump({
        'user_tokenized': user_res,
        'assistant_tokenized': assistant_res
    }, f, ensure_ascii=False, indent=2)

In [11]:
#合并两个词表
all_words = []
for i in tqdm(range(len(user_res)), desc="Merging words"):
    all_words += user_res[i] + assistant_res[i]

all_words[0]

Merging words: 100%|██████████| 110000/110000 [00:04<00:00, 27357.25it/s]


'能'

In [12]:
#去重，计算词表大小
all_words = sorted(set(all_words))
vocab_size = len(all_words)
vocab_size

490092

In [13]:
#构建索引映射
vocab = {token :index for index , token in enumerate(all_words)}
for i , item in enumerate(vocab.items()):
    if i >=50:
        break
    print(item)



('\x01', 0)
('\x08', 1)
('\t', 2)
('\n', 3)
('\x0c', 4)
('\r', 5)
(' ', 6)
('!', 7)
('"', 8)
('#', 9)
('##', 10)
('###', 11)
('####', 12)
('#####', 13)
('######', 14)
('##_', 15)
('##_##', 16)
('##__', 17)
('#%', 18)
('#&', 19)
('#+', 20)
('#++', 21)
('#-', 22)
('#.', 23)
('#.##', 24)
('#__', 25)
('$', 26)
('%', 27)
('%%', 28)
('%%%', 29)
('%&', 30)
('%+', 31)
('%-', 32)
('%.', 33)
('%.%', 34)
('%._', 35)
('%_', 36)
('&', 37)
('&#', 38)
('&#...', 39)
('&&', 40)
('&&...', 41)
('&+', 42)
('&-', 43)
('&--', 44)
('&--#', 45)
('&.', 46)
('&...', 47)
('&_', 48)
('&__', 49)


## Step3 构建简单分词器

In [14]:
class SimpleTokenizer():
    def __init__(self,vocab,tokenizer):
        self.str_to_int = vocab
        self.int_to_str = {val:key for key , val in vocab.items()}
        self.tokenizer = tokenizer
    def encoder(self,sequence):
        cut_seq = list(self.tokenizer.cut(sequence))
        result = []
        for i in range(len(cut_seq)):
            result.append(self.str_to_int[cut_seq[i]])
        return result
    def decoder(self,tokenized_result):
        result = ""
        for i in range(len(tokenized_result)):
            result += self.int_to_str[tokenized_result[i]]
        return result


In [15]:
user_list[0]

'能给我讲一个寓意深刻的故事吗？'

In [16]:
simple_tokenizer = SimpleTokenizer(vocab,tokenizer)
encoded = simple_tokenizer.encoder(user_list[0])
encoded

[421098,
 412376,
 312643,
 440095,
 178785,
 285388,
 369614,
 391059,
 328683,
 251846,
 489130]

In [17]:
decoded = simple_tokenizer.decoder(encoded)
decoded

'能给我讲一个寓意深刻的故事吗？'

## Step5 复杂的情况及分词器的改进

In [19]:
text = "oh！jjy，童话里做英雄！oh，jjy，热血心中流动"
encoded = simple_tokenizer.encoder(text)
encoded
#很多情况，构建词表还是会存在切词后词不在词表中的情况
#并且在实际nlp任务处理文本的时候，还会在开始，结尾处增加特殊标记

KeyError: 'jjy'

In [20]:
decoded = simple_tokenizer.decoder(encoded)
decoded

'能给我讲一个寓意深刻的故事吗？'

In [21]:
#增加特殊标记的vocab
vocab_len = len(vocab.keys()) # 490092
vocab["<bos>"] = 490092
vocab["<eos>"] = 490093
vocab["<unk>"] = 490094

In [30]:
#新的切词器
class NewTokenizer():
    def __init__(self,vocab,tokenizer):
        self.str_to_int = vocab
        self.int_to_str = {val:key for key , val in vocab.items()}
        self.tokenizer = tokenizer
    def encoder(self,sequence):
        cut_seq = list(self.tokenizer.cut(sequence))
        result = []
        result.append(self.str_to_int["<bos>"])
        for i in range(len(cut_seq)):
            if cut_seq[i] in self.str_to_int.keys():
                result.append(self.str_to_int[cut_seq[i]])
            else:
                result.append(self.str_to_int["<unk>"])
        result.append(self.str_to_int["eos"])
        return result
    def decoder(self,encoded):
        result = ""
        for i in range(len(encoded)):
            result += self.int_to_str[encoded[i]]
        return result
            

In [31]:
new_tokenizer = NewTokenizer(vocab,tokenizer)

In [32]:
text = "oh！jjy，童话里做英雄！oh，jjy，热血心中流动"
encoded = new_tokenizer.encoder(text)
encoded

[490092,
 154009,
 489102,
 490094,
 489111,
 403248,
 462325,
 214026,
 428341,
 489102,
 154009,
 489111,
 490094,
 489111,
 376565,
 305885,
 366869,
 134573]

In [33]:
decoded = new_tokenizer.decoder(encoded)
decoded

'<bos>oh！<unk>，童话里做英雄！oh，<unk>，热血心中流动eos'

## Step5 BPE分词器

In [34]:
"""
使用jieba分词会导致大规模的词模型没见过，从而无法准确完成encoder和decoder的任务
因此使用BPE分词
"""
import tiktoken

In [35]:
tokenizer = tiktoken.get_encoding("gpt2")#初始化GPT2!

In [36]:
text = "oh,jjy!童话里做英雄！oh，jjy，热血心中流动！"
encoded = tokenizer.encode(text)
encoded

[1219,
 11,
 41098,
 88,
 0,
 44165,
 98,
 46237,
 251,
 34932,
 234,
 161,
 223,
 248,
 164,
 233,
 109,
 37239,
 226,
 171,
 120,
 223,
 1219,
 171,
 120,
 234,
 41098,
 88,
 171,
 120,
 234,
 163,
 225,
 255,
 26193,
 222,
 33232,
 225,
 40792,
 38184,
 223,
 27950,
 101,
 171,
 120,
 223]

In [37]:
decoded = tokenizer.decode(encoded)
decoded

'oh,jjy!童话里做英雄！oh，jjy，热血心中流动！'

In [39]:
user_list[0]


'能给我讲一个寓意深刻的故事吗？'

## Step6 滑动窗口预测

In [42]:
encoded = tokenizer.encode(user_list[0])
len(encoded)

32

In [43]:
context = 4#上下文长度
first = encoded[:context]
second = encoded[1:context+1]
first , second

([47797, 121, 163, 119], [121, 163, 119, 247])

## Step7 构建GPT数据集

In [63]:
class GPTDataset(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        super().__init__()
        self.input_ids = []
        self.target_ids = []

        #生成encoded
        encoded = tokenizer.encode(txt)

        #生成数据
        for i in range(0,len(encoded)-max_length,stride):
            input = encoded[i:i+max_length]
            target = encoded[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input))
            self.target_ids.append(torch.tensor(target))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, index):
        return self.input_ids[index],self.target_ids[index]
 

In [64]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDataset(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [65]:
assistant_list[0]
dataloader = create_dataloader_v1(#raw_text 中创建一个数据加载器 但是所批次
    assistant_list[0], batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)#数据加载器 dataloader 转换为一个迭代器
first_batch = next(data_iter)
print(first_batch)

[tensor([[37605,   241, 47078,   114]]), tensor([[  241, 47078,   114, 20998]])]


## Step8 理解embedding层工作原理

In [85]:
"""
本质上就是一个one-hot编码+线性层
"""
input_ids = [1,2,6,4]
vocab_size = 7
output_dim = 3
class Embedding(nn.Module):
    def __init__(self,vocab_size,output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,output_dim)
    def forward(self,input):
        return self.embedding(input)
embedding = Embedding(vocab_size,output_dim)
embedded = embedding(torch.tensor(input_ids))
embedded


tensor([[-0.8618, -0.6245,  0.3002],
        [ 0.4662, -1.0248, -1.1778],
        [ 0.1495, -0.9649,  0.6886],
        [ 0.2623,  0.9774,  0.2025]], grad_fn=<EmbeddingBackward0>)

## Step9 位置编码与最终输入向量的生成

In [83]:
context_length = 4
pos_embedding = torch.nn.Embedding(context_length,output_dim)
pos_embeded = pos_embedding(torch.arange(context_length))
pos_embeded

tensor([[-0.4482,  0.5212,  1.8515],
        [-0.3122, -1.0258,  0.0761],
        [ 0.4272, -0.3092, -0.5890],
        [ 0.0937, -1.4066,  1.5259]], grad_fn=<EmbeddingBackward0>)

In [84]:
input_vec = pos_embeded + embedded
input_vec

tensor([[-1.2075, -0.8381,  0.6974],
        [-1.3548, -0.9634, -2.0201],
        [-0.3824, -0.8803, -1.4132],
        [ 0.4663, -0.6016,  1.4532]], grad_fn=<AddBackward0>)